<a href="https://colab.research.google.com/github/AdityaKumbhar21/Natural_Language_Processing/blob/main/Bidirectional_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Embedding
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [53]:
df_fake = pd.read_csv('/content/Fake.csv', on_bad_lines='skip', engine='python')
df_true = pd.read_csv('/content/True.csv',on_bad_lines='skip', engine='python')

In [54]:
print(f"Fake: {df_fake.shape}\nTrue: {df_true.shape}")

Fake: (23481, 4)
True: (21417, 4)


In [55]:
df_fake['label'] = 0
df_fake.head(10)

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [56]:
df_true['label'] = 1
df_true.head(10)

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017",1
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",politicsNews,"December 29, 2017",1
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,politicsNews,"December 29, 2017",1
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,politicsNews,"December 29, 2017",1
9,Alabama official to certify Senator-elect Jone...,WASHINGTON (Reuters) - Alabama Secretary of St...,politicsNews,"December 28, 2017",1


In [57]:
df_merged = pd.concat([df_fake, df_true], axis = 0)
df_merged


,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1


In [58]:
df_merged = df_merged.sample(frac=1)

In [59]:
df_merged.shape

(44898, 5)

In [60]:
df = df_merged.drop(['text','subject','date'], axis=1)
df.head(10)

,title,label
20590,Togo leader must quit now for protests to stop...,1
6368,Fast-food workers protest Trump's labor secret...,1
1459,Trump Gave HIGHLY Classified Information To R...,0
1311,Racist Jerk Takes His Hate To National Black ...,0
14415,Israeli minister reveals covert contacts with ...,1
8089,Trump Jr. tweet likening Syrian refugees to po...,1
22292,PROVOCATION? Republican Senators Introduce New...,0
22501,"Boiler Room EP #70 – Sticks, Stones & The Medi...",0
10060,"Republican Cruz crushes Trump in Wisconsin, sa...",1
22940,Billionaire ‘Bilderberger’ David Rockefeller d...,0


In [61]:
df_manual_testing = df.tail(10)
df = df.iloc[:-10]

In [64]:
X = df['title']
y = df['label']

In [65]:
voc_size = 5000

In [66]:
messages = X.copy()

In [67]:
messages.head()

,title
20590,Togo leader must quit now for protests to stop...
6368,Fast-food workers protest Trump's labor secret...
1459,Trump Gave HIGHLY Classified Information To R...
1311,Racist Jerk Takes His Hate To National Black ...
14415,Israeli minister reveals covert contacts with ...


In [69]:
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
  review = re.sub('[^a-zA-Z]', ' ',messages.iloc[i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [70]:
corpus[2]

'trump gave highli classifi inform russian ambassador right fire comey'

In [73]:
one_hot_repr = [one_hot(words, voc_size) for words in corpus]

In [74]:
max(len(sentence.split()) for sentence in corpus)

35

In [75]:
max_sent_length = 40
embedded_docs = pad_sequences(one_hot_repr, padding='pre',maxlen =max_sent_length)
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1604,
       3881,  352, 1423,   99, 1469, 1831, 2889], dtype=int32)

In [76]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [77]:
X_train,X_test, y_train,y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=0)

In [81]:
# creating a model
embedding_vector_feature = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_feature, input_length=max_sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [83]:
model.fit(X_train,y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
562/562 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9555 - loss: 0.1156 - val_accuracy: 0.9481 - val_loss: 0.1361
Epoch 2/10
562/562 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.9712 - loss: 0.0799 - val_accuracy: 0.9485 - val_loss: 0.1411
Epoch 3/10
562/562 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9760 - loss: 0.0693 - val_accuracy: 0.9442 - val_loss: 0.1651
Epoch 4/10
562/562 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.9816 - loss: 0.0537 - val_accuracy: 0.9436 - val_loss: 0.1847
Epoch 5/10
562/562 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.9864 - loss: 0.0415 - val_accuracy: 0.9404 - val_loss: 0.1974
Epoch 6/10
562/562 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.9884 - loss: 0.0341 - val_accuracy: 0.9423 - val_loss: 0.1865
Epoch 7/10
562/562 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.9890 - loss: 0.0330 - val_accuracy: 0.9405 - val_loss: 0.2341
Epoch 8/10
562/562 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9927 - loss: 0.0245 - val_accur

In [84]:
y_pred = model.predict(X_test)

281/281 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


In [85]:
y_pred = np.where(y_pred > 0.5, 1, 0)

In [86]:
print(confusion_matrix(y_test, y_pred))

[[4499  208]
 [ 342 3929]]


In [87]:
print(accuracy_score(y_test, y_pred))

0.938739140120294


In [88]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.96      0.94      4707
           1       0.95      0.92      0.93      4271

    accuracy                           0.94      8978
   macro avg       0.94      0.94      0.94      8978
weighted avg       0.94      0.94      0.94      8978

